In [29]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [30]:
base_url = 'https://www.4icu.org/in/universities/'
req = requests.get(base_url)
soup = BeautifulSoup(req.content, 'html5lib')

Defining a function named 'state_url' which takes base website url as argument and returns the list of all the urls of the state.

In [31]:
def state_url(base_url):
  state_content = soup.find_all('div', class_ = 'panel-body')
  base_url_ = 'https://www.4icu.org'
  list_of_state_urls = []
  for each in state_content:
    s = each.find_all('tr')
    for each1 in s:
         a = each1.find_all('a')
         for each2 in a:
           url_ = each2.get('href') 
           url_ = base_url_ + url_
           list_of_state_urls.append(url_)
  return list_of_state_urls
State_urls = state_url(base_url)
#State_urls

Defining a function named 'coll_urls' which take state url as argument and returns the list of all the college urls in a list.

In [32]:
def coll_urls(state_url):
  base_url_ = 'https://www.4icu.org'
  req = requests.get(state_url)
  soup = BeautifulSoup(req.content, 'html5lib')
  url_content = soup.find_all('div', class_ = 'panel-body')
  list_of_college_urls = []
  for each in url_content:
    s = each.find_all('tr')
    for each1 in s:
         a = each1.find_all('a')
         for each2 in a:
           url_ = each2.get('href')
           url_ = base_url_ + url_
           list_of_college_urls.append(url_)
  return list_of_college_urls[7:-1]
State_wise_coll_urls =coll_urls('https://www.4icu.org/in/kerala/')  
#State_wise_coll_urls

In [33]:
capital_cities = ['Amaravati', 'Itanagar', 'Dispur', 'Patna', '	Raipur', 'Panaji',
                 'Gandhinagar', 'Chandigarh', 'Shimla', 'Srinagar', 'Ranchi', 'Bengaluru',
                 'Trivandrum','Bhopal', 'Mumbai', 'Imphal', 'Shillong', 'Aizawl', 'Kohima',
                 'Bhubaneswar', 'Chandigarh', 'Jaipur', 'Gangtok', 'Chennai', 'Hyderabad',
                 'Agartala', 'Lucknow', 'Dehradun', 'Kolkata']

In [34]:
def capital_cities_colleges_url(url):
    base_url_ = 'https://www.4icu.org'
    req = requests.get(state_url)
    soup = BeautifulSoup(req.content, 'html5lib')
    url_content = soup.find_all('div', class_ = 'panel-body')
    list = []
    for each in url_content:
      s = each.find_all('tr')
      for each1 in s:
          a = each1.find_all('a')
          td = each1.find_all('td')
          if len(td) == 3:
            town = td[-1].text
            if town in capital_cities:
               for each2 in a:
                    url_ = each2.get('href')
                    url_ = base_url_ + url_
                    list.append(url_)
    return list

In [35]:
metro_cities = ['Mumbai', 'Delhi', 'Kolkata', 'Lucknow']

In [36]:
def metro_cities_colleges_url(url):
    base_url_ = 'https://www.4icu.org'
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html5lib')
    url_content = soup.find_all('div', class_ = 'panel-body')
    list = []
    for each in url_content:
      s = each.find_all('tr')
      for each1 in s:
          a = each1.find_all('a')
          td = each1.find_all('td')
          if len(td) == 3:
            town = td[-1].text
            if town in metro_cities:
               for each2 in a:
                    url_ = each2.get('href')
                    url_ = base_url_ + url_
                    list.append(url_)
    return list

In [37]:

def college_details(url):
  req = requests.get(url)
  soup = BeautifulSoup(req.content, 'html5lib')

  dict_ = {}

  # College name
  name_ = soup.find('ol', class_ = 'breadcrumb small')
  dict_['College Name'] = name_.find_all('span', itemprop = 'name')[-1].text

  # College url
  url_ = soup.find_all('a', itemprop="url")
  dict_['College url'] = url_[1].get('href')

  # College Address
  add_1 = soup.find_all('span', itemprop = 'streetAddress')[0].text
  add_2 = soup.find_all('span', itemprop = 'addressLocality')[0].text
  add_3 = soup.find_all('span', itemprop = 'postalCode')[0].text
  add_4 = soup.find_all('span', itemprop = 'addressRegion')[0].text
  add = add_1 + ' ' + add_2 + add_3 + ' ' + add_4
  dict_['College Address'] = add

  # College Contact Number
  contact_no = soup.find_all('span', itemprop = 'telephone')[0].text
  dict_['Contact no.'] = contact_no

  return dict_

In [39]:
print('Enter 1, for the details of all the top colleges in India')
print('Enter 2, for all the colleges in capital cities of India')
print('Enter 3, for all the colleges in metro cities of India')

input_ = int(input())
if input_ == 1:
  colleges_list = []
  for each in State_urls:
      colleges = coll_urls(each)
      for each1 in colleges:
         colleges_list.append(college_details(each1))
  df = pd.DataFrame(colleges_list)
  df.to_csv('all_colleges.csv')
elif input_ == 2:
  colleges_list = []
  for each in State_urls:
      colleges = capital_cities_colleges_url(each)
      for each1 in colleges:
         colleges_list.append(college_details(each1))
  df = pd.DataFrame(colleges_list)
  df.to_csv('capital_colleges.csv')
elif input_ == 3:
  colleges_list = []
  for each in State_urls:
      colleges = metro_cities_colleges_url(each)
      for each1 in colleges:
         colleges_list.append(college_details(each1))
  df = pd.DataFrame(colleges_list)
  df.to_csv('metrocities_colleges.csv')
else:
  print('Invalid Input')

Enter 1, for the details of all the top colleges in India
Enter 2, for all the colleges in capital cities of India
Enter 3, for all the colleges in metro cities of India
3
